In [85]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [86]:
df_train = pd.read_csv('Training.csv')
df_test = pd.read_csv('Testing.csv')

In [87]:
df_train= df_train.drop('Unnamed: 133',axis=1)

In [88]:
df_train.head()

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
1,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
2,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
3,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
4,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection


In [89]:
df_test.head()

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
1,0,0,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Allergy
2,0,0,0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,GERD
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Chronic cholestasis
4,1,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Drug Reaction


In [90]:
df_train.shape,df_test.shape

((4920, 133), (42, 133))

In [91]:
X_train = df_train.drop('prognosis',axis=1)
X_test = df_test.drop('prognosis',axis=1)

y_train = df_train['prognosis']
y_test = df_test['prognosis']

In [92]:
y_train = pd.DataFrame(y_train)
y_test = pd.DataFrame(y_test)

In [93]:
y_train.head()

,prognosis
0,Fungal infection
1,Fungal infection
2,Fungal infection
3,Fungal infection
4,Fungal infection


In [94]:
## Onehot encode 'Geography
from sklearn.preprocessing import OneHotEncoder
onehot_encoder=OneHotEncoder()
y_trainOHE=onehot_encoder.fit_transform(y_train[['prognosis']]).toarray()
y_testOHE=onehot_encoder.fit_transform(y_test[['prognosis']]).toarray()

In [95]:
import pickle

with open('onehot_encoder.pkl', 'wb') as f:
    pickle.dump(onehot_encoder, f)

In [97]:
y_testOHE = pd.DataFrame(y_testOHE)
y_trainOHE = pd.DataFrame(y_trainOHE)

ANN Implementation

In [98]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [99]:
(X_train.shape[1],)

(132,)

In [112]:
from tensorflow.keras.layers import Dropout

model = Sequential([
    Dense(84, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(42, activation='relu'),
    Dropout(0.3),
    Dense(20, activation='relu'),
    Dropout(0.3),
    Dense(41, activation='softmax')
])

c:\Projects\diesease prediction\devm\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [113]:
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.CategoricalCrossentropy()

In [114]:
## compile the model
model.compile(optimizer=opt,loss="categorical_crossentropy",metrics=['binary_accuracy','accuracy','categorical_accuracy'])

In [115]:
## Set up the Tensorboard
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [116]:
## Set up Early Stopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=15,restore_best_weights=True)

In [117]:
### Train the model
history=model.fit(
    X_train,y_trainOHE,validation_data=(X_test,y_testOHE),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100


154/154 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.3142 - binary_accuracy: 0.9789 - categorical_accuracy: 0.3142 - loss: 2.4303 - val_accuracy: 0.9524 - val_binary_accuracy: 0.9971 - val_categorical_accuracy: 0.9524 - val_loss: 0.1518
Epoch 2/100
154/154 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7152 - binary_accuracy: 0.9876 - categorical_accuracy: 0.7152 - loss: 0.8669 - val_accuracy: 1.0000 - val_binary_accuracy: 0.9994 - val_categorical_accuracy: 1.0000 - val_loss: 0.0495
Epoch 3/100
154/154 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7584 - binary_accuracy: 0.9893 - categorical_accuracy: 0.7584 - loss: 0.7187 - val_accuracy: 0.9762 - val_binary_accuracy: 0.9994 - val_categorical_accuracy: 0.9762 - val_loss: 0.0429
Epoch 4/100
154/154 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7851 - binary_accuracy: 0.9906 - categorical_accuracy: 0.7851 - loss: 0.6425 - val_accuracy: 0.9762 - val_binary_accuracy: 0.9994 - val_categorical_accuracy: 0.9762 - val_loss: 0.0629
Epoch 5/

In [118]:
model.save('ANN_model.h5')

In [119]:
## Load Tensorboard Extension
%load_ext tensorboard

In [121]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 8704), started 0:00:04 ago. (Use '!kill 8704' to kill it.)

In [123]:
model.evaluate(X_test, y_testOHE)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9841 - binary_accuracy: 0.9996 - categorical_accuracy: 0.9841 - loss: 0.0320 


[0.04290388897061348, 0.9994192719459534, 0.976190447807312, 0.976190447807312]